This project was developed by Linda Mendez, a student at the Universidad Nacional de Colombia. 


This notebook introduces a musical box, explains the implementation with configurability, and generates a tapeout-ready(GDS) design with open-source tools and PDK.

The musical box was a project developed with peripherals such as a disk, keys that represent an octave in the piano, buttons to control the volume, and the option to play freely or record the melodies played on the keys to be played when the user wants. The complete project was implemented in VHDL, but only the keyboard was implemented in Verilog and used in this notebook.


# Introduction

This project was designed for use by blind people, so the musical notes are written in Braille on the keyboard. The motivation is to provide a tool for blind people to play their melodies and listen to them. 
This program has as input an array of keys, where each time a key is pressed, its bit changes to one, the simulation also receives the time that this key was pressed since this is a physical input variable. In the physical implementation, the musical notes were written in braille.

![bconfig](https://github.com/LindaM123/caja_musica/blob/main/Images/pianito.png?raw=true)





# How does it work?

The music box receives the key that is pressed, it also has a reset button. When a key is pressed, the output of the box is a signal that has the frequency of the chosen note, to produce signals with different frequencies frequency dividers are used.


![bconfig](https://github.com/LindaM123/caja_musica/blob/main/Images/state_machine.png?raw=true)



In [35]:
!rm -rf caja_musica
!git clone https://github.com/LindaM123/caja_musica.git
!rm -rf musica
!mv caja_musica/musica .
!rm -rf caja_musica

Cloning into 'caja_musica'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 83 (delta 32), reused 65 (delta 20), pack-reused 0
Unpacking objects: 100% (83/83), 6.22 MiB | 9.41 MiB/s, done.


In [ ]:
!apt-get install verilator
!pip3 install fxpmath

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libsystemc libsystemc-dev
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  libsystemc libsystemc-dev verilator
0 upgraded, 3 newly installed, 0 to remove and 23 not upgraded.
Need to get 5,212 kB of archives.
After this operation, 24.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libsystemc amd64 2.3.3-4build1 [449 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libsystemc-dev amd64 2.3.3-4build1 [238 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 verilator amd64 4.028-1 [4,525 kB]
Fetched 5,212 kB in 0s (12.2 MB/s)
Selecting previously unselected package libsystemc:amd64.
(Reading database ... 128285 files and directories currently installed.)
Preparing to unpack .../libsystemc_2.3.3-4build1_amd64.deb ...
Unpacking libs

In [3]:
#@title Install dependencies {display-mode: "form"}
#@markdown - Click the ▷ button to setup the digital design environment based on [conda-eda](https://github.com/hdl/conda-eda).

openlane_version = 'latest' #@param {type:"string"}
open_pdks_version = 'latest' #@param {type:"string"}

if openlane_version == 'latest':
  openlane_version = ''
if open_pdks_version == 'latest':
  open_pdks_version = ''



import os
import pathlib

!curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xj bin/micromamba
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
!bin/micromamba create --yes --prefix $CONDA_PREFIX
!echo 'python ==3.7*' >> {CONDA_PREFIX}/conda-meta/pinned
!CI=0 bin/micromamba install --yes --prefix $CONDA_PREFIX \
                     --channel litex-hub \
                     --channel main \
                     openlane={openlane_version} \
                     open_pdks.sky130a={open_pdks_version} 
!python -m pip install gdstk gdstk
PATH = os.environ['PATH']
%env CONDA_PREFIX={CONDA_PREFIX}
%env PATH={CONDA_PREFIX}/bin:{PATH}


                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/

Empty environment created at prefix: /content/conda-env

                                           __
          __  ______ ___  ____ _____ ___  / /_  ____ _
         / / / / __ `__ \/ __ `/ __ `__ \/ __ \/ __ `/
        / /_/ / / / / / / /_/ / / / / / / /_/ / /_/ /
       / .___/_/ /_/ /_/\__,_/_/ /_/ /_/_.___/\__,_/
      /_/


Pinned packages:
  - python 3.7*


Transaction

  Prefix: /content/conda-env

  Updating specs:

   - openlane=*
   - open_pdks.sky130a=*


  Package                              Version  Build                 Channel                 Size
────────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
──────────────────────────────────────────

In [4]:
%%writefile config.json
{
    "DESIGN_NAME": "principal",
    "VERILOG_FILES": "dir::musica/rtl/principal.v",
    "CLOCK_PERIOD": 20,
    "CLOCK_NET": "i_clk",
    "CLOCK_PORT": "i_clk",

    "FP_SIZING": "absolute",
    "DIE_AREA": "0 0 300 300"
}

Writing config.json


In [5]:
%env PDK=sky130A
!flow.tcl -design .

env: PDK=sky130A
OpenLane 2023.03.01_0_ge10820ec-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.json'...
[INFO]: PDK Root: /content/conda-env/share/pdk
[INFO]: Process Design Kit: sky130A
[INFO]: Standard Cell Library: sky130_fd_sc_hd
[INFO]: Optimization Standard Cell Library: sky130_fd_sc_hd
[INFO]: Run Directory: /content/runs/RUN_2023.04.03_03.04.52
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[STEP 1]
[INFO]: Running Synthesis (log: runs/RUN_2023.04.03_03.04.52/logs/synthesis/1-synthesis.log)...
[STEP 2]
[INFO]: Running Single-Corner Static Timing Analysis (log: runs/RUN_2023.04.03_03.04.52/logs/synthesis/2-sta.log)...
[STEP 3]
[INFO]: Running Initial Floorplanning (log: runs/RUN_2023.04.03_03.04.52/logs/floorplan/3-ini

In [12]:
import pathlib
import gdstk

gdss = sorted(pathlib.Path('runs').glob('*/results/final/gds/*.gds'))
library = gdstk.read_gds(gdss[-1])
top_cells = library.top_level()
top_cells[0].write_svg('principal.svg')

In [13]:
!pip install svgutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from IPython.display import SVG,display

display(SVG(filename='principal.svg'))

In [15]:
import pandas as pd
import pathlib

pd.options.display.max_rows = None
final_summary_reports = sorted(pathlib.Path('runs').glob('*/reports/metrics.csv'))
df = pd.read_csv(final_summary_reports[-1])
df.transpose()

,0
design,/content
design_name,principal
config,RUN_2023.04.03_03.04.52
flow_status,flow completed
total_runtime,0h3m6s0ms
routed_runtime,0h2m23s0ms
(Cell/mm^2)/Core_Util,4111.111111
DIEAREA_mm^2,0.09
CellPer_mm^2,2055.555556
OpenDP_Util,2.42


# Simulation

In [17]:
!apt-get install iverilog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  iverilog
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 1,859 kB of archives.
After this operation, 6,547 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 iverilog amd64 10.3-1build1 [1,859 kB]
Fetched 1,859 kB in 2s (796 kB/s)
Selecting previously unselected package iverilog.
(Reading database ... 128288 files and directories currently installed.)
Preparing to unpack .../iverilog_10.3-1build1_amd64.deb ...
Unpacking iverilog (10.3-1build1) ...
Setting up iverilog (10.3-1build1) ...
Processing triggers for man-db (2.9.1-1) ...


In [36]:
import random

data_amount = 100

binary_keys = ["0000000","1000000","0100000","0010000", "0001000", "0000100","0000010","0000001" ]
frequencies = ["0", "1975", "1760", "1567", "1396", "1318", "1174","1046"]    

f_data = open ('datos_prueba.txt','w')
f_frequencies = open ('frequencies_test.txt','w')

#musical_notes = ["Silence", "Si","La", "Sol", "Fa", "Mi", "Re", "Do"]

n = data_amount

keys_binary_test = [0]  * n
frequencies_test = [0]  * n

for i in range(n):
  number = random.randint(0, 7)
  keys_binary_test[i] = binary_keys[number]
  frequencies_test[i] = frequencies[number]        
  f_data.write(keys_binary_test[i] + "\n") 
  f_frequencies.write(frequencies_test[i] + "\n")
  frequencies_test[i] = int(frequencies_test[i])
      

#print("Ingrese cuantos numeros aleatorios desea obtener")
#n=int(input())



print(keys_binary_test)

f_frequencies.close()
f_data.close()

['0000001', '0010000', '1000000', '0000000', '0000100', '0001000', '0000010', '0000001', '0000000', '0000001', '0100000', '0000001', '0000100', '0100000', '0100000', '0000001', '0000000', '0100000', '1000000', '0000100', '0000000', '0001000', '0000100', '0010000', '0010000', '0100000', '0000000', '0010000', '0000001', '0000000', '1000000', '0100000', '0000000', '0000000', '0000010', '0100000', '0000100', '0000000', '0100000', '1000000', '0001000', '0000000', '1000000', '0000000', '0000000', '0000000', '0000100', '0000100', '0010000', '1000000', '0000000', '0010000', '0000001', '1000000', '0001000', '0000100', '0000001', '1000000', '0000001', '0001000', '0000001', '0100000', '0010000', '1000000', '0000000', '0000001', '0100000', '1000000', '0100000', '1000000', '0000010', '0100000', '1000000', '0000000', '0001000', '1000000', '0000000', '0010000', '0000001', '0000001', '0100000', '1000000', '1000000', '0000000', '0000100', '0010000', '0000010', '0000010', '1000000', '0000001', '0001000'

In [37]:
f_times = open ('tiempos_prueba.txt','w')

musical_notes = ["Silence", "Si","La", "Sol", "Fa", "Mi", "Re", "Do"]

n = data_amount

test_times = [0]  * n

for i in range(n):
      number = random.randint(2000000,10000000)
      number_bin = bin(number)[2:]
  
      print(number)
      print(number_bin)

      test_times[i] = number_bin
      f_times.write(test_times[i] + "\n")

#print("Ingrese cuantos numeros aleatorios desea obtener")
#n=int(input())

print(test_times)

f_times.close()

9222120
100011001011011111101000
5047386
10011010000010001011010
3157180
1100000010110010111100
8051398
11110101101101011000110
8752527
100001011000110110001111
5255135
10100000010111111011111
6956090
11010100010010000111010
9184682
100011000010010110101010
7740366
11101100001101111001110
7633671
11101000111101100000111
2591472
1001111000101011110000
9420376
100011111011111001011000
5376125
10100100000100001111101
2721954
1010011000100010100010
2980667
1011010111101100111011
6823788
11010000001111101101100
5650652
10101100011100011011100
6834981
11010000100101100100101
9026805
100010011011110011110101
3769726
1110011000010101111110
7916647
11110001100110001100111
4984288
10011000000110111100000
9259650
100011010100101010000010
3701597
1110000111101101011101
4889697
10010101001110001100001
6288137
10111111111001100001001
3231107
1100010100110110000011
3491816
1101010100011111101000
3183389
1100001001001100011101
6116121
10111010101001100011001
9080567
100010101000111011110111
3657312
11

In [38]:
!iverilog -o sim_principal musica/test/sim_principal.v musica/rtl/principal.v  

In [39]:
!vvp sim_principal

VCD info: dumpfile sim_principal.vcd opened for output.
              956040
                   9
              338280
              638200
                   7
               58214
              506360
                   6
              387340
          0
              758760
                  11
              352593
              716360
                   7
              475745
              851800
                   8
              710110
              956040
                   9
              375718
          0
              956040
                   7
               14649
              568200
                   4
              249528
              956040
                   9
              140024
              758760
                   7
              693955
              568200
                   4
              119046
              568200
                   5
              428533
              956040
                   7
              824532
          0
              568200
     

# Comparación resultados simulacion

In [40]:
file_out = open('output.txt','r')

measured_frequencies = [0] * data_amount

lines = file_out.readlines()
i = 0
for line in lines:
   number = int(line) * pow(10,-12)
   if number != 0 :
    measured_frequencies[i] = 1/number
    print(measured_frequencies[i])
   else: 
        measured_frequencies[i] = number
        print(measured_frequencies[i])
   i+=1
file_out.close()


1045.9813396928998
1566.9069257286117
1974.879532348527
0.0
1317.9397965100954
1395.9461723155955
1173.984503404555
1045.9813396928998
0.0
1045.9813396928998
1759.9436818021823
1045.9813396928998
1317.9397965100954
1759.9436818021823
1759.9436818021823
1045.9813396928998
0.0
1759.9436818021823
1974.879532348527
1317.9397965100954
0.0
1395.9461723155955
1317.9397965100954
1566.9069257286117
1566.9069257286117
1759.9436818021823
0.0
1566.9069257286117
1045.9813396928998
0.0
1974.879532348527
1759.9436818021823
0.0
0.0
1173.984503404555
1759.9436818021823
1317.9397965100954
0.0
1759.9436818021823
1974.879532348527
1395.9461723155955
0.0
1974.879532348527
0.0
0.0
0.0
1317.9397965100954
1317.9397965100954
1566.9069257286117
1974.879532348527
0.0
1566.9069257286117
1045.9813396928998
1974.879532348527
1395.9461723155955
1317.9397965100954
1045.9813396928998
1974.879532348527
1045.9813396928998
1395.9461723155955
1045.9813396928998
1759.9436818021823
1566.9069257286117
1974.879532348527
0.0
1

# Total Error in the musical box

In [43]:
import math
errors_abs = [0] * data_amount
sum = 0
for i in range(data_amount):
  if(frequencies_test[i] != 0):
    errors_abs[i] = abs(measured_frequencies[i] - frequencies_test[i])/frequencies_test[i]
    #print(errors_abs[i])
    sum += pow(frequencies_test[i] - measured_frequencies[i] , 2)
    sum = sum/data_amount

print(sum)


3.200751597477e-05


# Improvements for next sessions

The initial project was built in VHDL and had more functions as a disk, volume buttons, and the option to record the melodies to listen to them later, so one way to complement the project implemented in this notebook would be to implement the other modules in Verilog to be included in this work.
A ring oscillator could be added to generate a clock inside the chip and not have to add a PAD.

